<a href="https://colab.research.google.com/github/Katiaaaa/NBC/blob/main/NBC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Naive Bayes Classifier

Welcome to your next lab! You will build Naive Bayes Classifier.

You will classify spam/ham messages.

**You will learn to:**
- Build the general architecture of a learning algorithm with OOP in mind:
    - Helper functions
        - Preprocessing data
    - Main Model Class
        - Training
        - Prediction 


## 1 - Packages ##

First, let's run the cell below to import all the packages that you will need during this assignment.
- [numpy](www.numpy.org) is the fundamental package for scientific computing with Python.
- [pandas](https://pandas.pydata.org/) is a library providing a convenient work with data.
- [re](https://docs.python.org/3/library/re.html) is for regex

In [1]:
import pandas as pd
import numpy as np
import re

## 2 - Overview of the Problem set ##

**Problem Statement**: You are given a dataset  containing:
    - a training set of m_train examples
    - a test set of m_test examples
    - each example is a message that belongs to a particular class: ham or spam.

Let's get more familiar with the dataset. Load the data by running the following code.

We won't divide our data to features(X) and target(Y) here, because we need to preprocess it in a special way.

In [2]:
# Loading the data 

def load_data():
    df = pd.read_csv('spam.csv', encoding='latin-1')
    df_for_tests = df.head()
    
    idx = np.arange(df.shape[0])
    np.random.shuffle(idx)

    train_set_size = int(df.shape[0] * 0.8)

    train_set = df.loc[idx[:train_set_size]]
    test_set = df.loc[idx[train_set_size:]]
    
    return train_set, test_set, df_for_tests

In [3]:
train_set, test_set, df_for_tests = load_data()

## 3 - Naive Bayes Classifier
**Mathematical expression of the algorithm**:


This algorithm is based on Bayes' theorem:
    \begin{equation}
    P(A_{j}\hspace{0.1cm}\rvert\hspace{0.1cm}x_{1},\dots,x_{n}) = \frac{P(x_{1},\dots,x_{n}\hspace{0.1cm}\rvert\hspace{0.1cm}A_{j})P(A_{j})}{P(x_{1},\dots,x_{n})} 
    \end{equation}
    
Ignoring denominator (because it stays the same for all cases):

$$ \begin{equation}
    P(A_{j})P(x_{1},\dots,x_{n}\hspace{0.1cm}\rvert\hspace{0.1cm}A_{j}) = P(A_{j}, x_{1},\dots,x_{n}) = \\
  \hspace{1cm} = P(x_{1}\hspace{0.1cm}\rvert\hspace{0.1cm}x_{2},\dots,x_{n}, A_{j})P(x_{2}\hspace{0.1cm}\rvert\hspace{0.1cm}x_{3}, \dots ,x_{n}, A_{j})\dots P(x_{n-1}\hspace{0.1cm}\rvert\hspace{0.1cm}x_{n}, A_{j}) \approx \\
  \hspace{1cm}
  \end{equation}$$
By making an assumption that the $x_{i}$ are conditionally independent of each other:
$$ \begin{equation} \approx P(A_{j}) \prod_{i=1}^{n} P(x_{i}\hspace{0.1cm}\rvert\hspace{0.1cm}A_{j})
   \end{equation}$$
   
We can calculate the probability, if we know the prior probability:

$$ \begin{equation}
    y^{*} = \operatorname*{arg\,max}_{j} \big(P(A_{j}) \prod_{i=1}^{n} P(x_{i}\hspace{0.1cm}\rvert\hspace{0.1cm}A_{j})\big)
   \end{equation}$$
   
   
Due to floating point underflow, the above is usually replaced with the numerically tractable expression:

$$ \begin{equation}
    y^{*} = \operatorname*{arg\,max}_{j} \big( \ln(P(A_{j})) + \sum_{i=1}^{n} \ln(P(x_{i}\hspace{0.1cm}\rvert\hspace{0.1cm}A_{j})) \big)
   \end{equation}$$
   
For more consistent knowledge of the NBC algorithm, we highly recommend you to read [this Stanford University article](https://web.stanford.edu/~jurafsky/slp3/4.pdf)   
   
#### Laplace smoothing

In statistics, additive smoothing, also called Laplace smoothing, or Lidstone smoothing, is a technique that is used to smooth categorical data. Given an observation 
$\begin{equation}
    x = (x_{1}\, \dots \,x_{k})
 \end{equation}$ from a multinomial distribution with N trials, a "smoothed" version of the data gives the estimator:

$$ \begin{equation}
    \theta_i = \frac{x_{i} + \alpha}{N + \alpha k}
   \end{equation}$$

where the pseudocount 
$\begin{equation}
    \alpha > 0
 \end{equation}$ is the smoothing parameter (
$\begin{equation}
    \alpha = 0
 \end{equation}$ corresponds to no smoothing).

### 3.1 - Preprocessing the data 

Our data consists of different messages. Messages contain some excess symbols, which don't affect the content of the text, but add noise to the data.
For example: "Does not \\operate 66.7 after  & lt;# & gt;  or what". 

Let's clean our data and leave only letters a-z, A-Z, numbers 0-9 and cast all letters to lowercase, replace double to n spaces with just one space, remove trailing spaces.

In [4]:
# Clean the data

def clean_data(message):
    
    """ 
    Returns string which consists of message words
    
    Argument:
    message -- message from dataset; 
        type(message) -> <class 'str'>
    
    Returns:
    result -- cleaned message, which contains only letters a-z, and numbers 0-9, with only one space between words;
        type(clean_data(message)) -> <class 'str'>
    
    """
    
    ### START CODE HERE ###
    return re.sub(r'\s+',' ' , re.sub('[^A-Za-z0-9]',' ',message)).lower()
    ### END CODE HERE ###

In [5]:
sentence_1 = 'Doesn\'t get, how{to}% \\operate+66.7 :after[it]"" & lt;# & gt; won\'t `or(what)'
sentence_2 = 'O\]k,.lar7i$double{} check wif*& da! hair: [dresser;   ..already He SaID-77.88.5 wun cut v short question(std txt rate)T&C\'s'
print('cleaned: ',clean_data(sentence_1))
print('cleaned: ',clean_data(sentence_2))

cleaned:  doesn t get how to operate 66 7 after it lt gt won t or what 
cleaned:  o k lar7i double check wif da hair dresser already he said 77 88 5 wun cut v short question std txt rate t c s


**Expected Output**: 

<table style="width:70%">
    <tr>
        <td><b>cleaned:</b></td>
       <td> doesn t get how to operate 66 7 after it lt gt won t or what </td>
    </tr>
    <tr>
        <td><b>cleaned:</b></td>
       <td> o k lar7i double check wif da hair dresser already he said 77 88 5 wun cut v short question std txt rate t c s </td>
    </tr>
    

</table>

Now let's clean each sentence and split data on features(X) and target(Y)

In [6]:
# Preparation data for model

def prep_for_model(train_set, test_set):
    
    """ 
    Returns arrays of train/test features(words) and train/test targets(labels)
    
    Arguments:
    train_set -- train dataset, which consists of train messages and labels; 
        type(train_set) -> pandas.core.frame.DataFrame
    test_set -- test dataset, which consists of test messages and labels; 
        type(train_set) -> pandas.core.frame.DataFrame
    
    Returns:
    train_set_x -- array which contains lists of words of each cleaned train message; 
        (type(train_set_x) ->numpy.ndarray[list[str]], train_set_x.shape = (num_messages,))
    train_set_y -- array of train labels (names of classes), 
        (type(train_set_y) -> numpy.ndarray, train_set_y.shape = (num_messages,))
    test_set_x -- array which contains lists of words of each cleaned test message;
        (type(test_set_x) numpy.ndarray[list[str]], test_set_x.shape = (num_messages,)
    test_set_y -- array of test labels (names of classes), 
        (type(test_set_y) -> numpy.ndarray, test_set_y.shape = (num_messages,))
    
    """
    
    ### START CODE HERE ###
    train_set_x=train_set['v2'][:]
    train_set_y=train_set['v1'][:]
    test_set_x=test_set['v2'][:]
    test_set_y=test_set['v1'][:]
    train_set_x = np.array( [  (clean_data(train_set_x[i])).split() for i in (train_set_x.index) ],dtype='object')
    train_set_y=np.array( [(clean_data(train_set_y[i])) for i in (train_set_y.index) ],dtype='object')
    test_set_x=np.array( [(clean_data(test_set_x[i])).split() for i in (test_set_x.index) ],dtype='object')
    test_set_y=np.array( [(clean_data(test_set_y[i])) for i in (test_set_y.index) ],dtype='object')

    return train_set_x, train_set_y, test_set_x, test_set_y

train_set_x, train_set_y, test_set_x, test_set_y = prep_for_model(train_set, test_set)

In [7]:
a1, a2, b1, b2 = prep_for_model(df_for_tests.head(3), df_for_tests.tail(2))
print(a2[0], a1[0])
print(b2[0], b1[0])

ham ['go', 'until', 'jurong', 'point', 'crazy', 'available', 'only', 'in', 'bugis', 'n', 'great', 'world', 'la', 'e', 'buffet', 'cine', 'there', 'got', 'amore', 'wat']
ham ['u', 'dun', 'say', 'so', 'early', 'hor', 'u', 'c', 'already', 'then', 'say']


**Expected Output**: 

<table style="width:40%">
    <tr>
        <td><b>ham:</b></td>
       <td> ['go', 'until', 'jurong', 'point', 'crazy', 'available', 'only', 'in', 'bugis', 'n', 'great', 'world', 'la', 'e', 'buffet', 'cine', 'there', 'got', 'amore', 'wat'] </td>
    </tr>
    <tr>
        <td><b>ham:</b></td>
       <td> ['u', 'dun', 'say', 'so', 'early', 'hor', 'u', 'c', 'already', 'then', 'say']
 </td>
    </tr>

</table>

Now let's check words in each category

In [8]:
# Check words in categories

def categories_words(x_train, y_train):

    """
    Returns arrays of features(words) in each category and in both categories

    Arguments:
    x_train -- array which contains lists of words of each cleaned train message;
        (type(x_train) -> numpy.ndarray[list[str]], x_train.shape = (num_messages,))

    Returns:
    all_words_list -- array of all words in both categories;
        (type(all_words_list) -> numpy.ndarray[str], all_words_list.shape = (num_words,))
    ham_words_list -- array of words in 'ham' class;
        (type(ham_words_list) -> numpy.ndarray[str], ham_words_list.shape = (num_words,))
    spam_words_list -- array of words in 'spam' class;
        (type(spam_words_list) -> numpy.ndarray[str], spam_words_list.shape = (num_words,))
    """
    all_words_list = []
    ham_words_list = []
    spam_words_list = []

    for i in range(x_train.size):
        all_words_list += x_train[i]
        if y_train[i] == 'ham':
            ham_words_list += x_train[i]
        else:
            spam_words_list += x_train[i]



    all_words_list= np.array(all_words_list)
    ham_words_list= np.array(ham_words_list)
    spam_words_list= np.array(spam_words_list)

    ### END CODE HERE ###

    return all_words_list, ham_words_list, spam_words_list

all_words_list_a1, ham_words_list_a1, spam_words_list_a1 = categories_words(a1, a2)

In [9]:
print('first five "ham" words of a1: ', ham_words_list_a1[:5])

first five "ham" words of a1:  ['go' 'until' 'jurong' 'point' 'crazy']


**Expected Output**: 

<table style="width:40%">
    <tr>
        <td><b>first five "ham" words of a1:</b></td>
       <td> ['go' 'until' 'jurong' 'point' 'crazy'] </td>
    </tr>

</table>

### 3.2 Model

In [10]:
class Naive_Bayes(object):
    """
    Parameters:
    -----------
    alpha: int
        The smoothing coeficient.
    """
    def __init__(self, alpha):
        self.alpha = alpha

        self.train_set_x = None
        self.train_set_y = None

        self.all_words_list = []
        self.ham_words_list = []
        self.spam_words_list = []

        self.ham_words_dict = {}
        self.spam_words_dict = {}

        self.prior_ham_prob = None
        self.prior_spam_prob = None

        #You are allowed to create new attributes and methods (but it's not a necessary)

    def fit(self, train_set_x, train_set_y):


        self.train_set_x = train_set_x
        self.train_set_y = train_set_y

        self.all_words_list, self.ham_words_list, self.spam_words_list =categories_words(train_set_x,train_set_y)

        self.prior_spam_prob=self.spam_words_list.size/self.all_words_list.size
        self.prior_ham_prob=1-self.prior_spam_prob
        y_size=train_set_y.size
        all_size=self.all_words_list.size
        self.all_words_list= np.unique(self.all_words_list)

        ham_size=self.ham_words_list.size
        spam_size=self.spam_words_list.size
        #print(self.prior_ham_prob,self.prior_spam_prob,self.ham_words_dict['asf'])
        for i in self.all_words_list:
            self.spam_words_dict[i]= np.log((np.count_nonzero(self.spam_words_list==i) +self.alpha)/(spam_size + self.alpha*all_size))
            self.ham_words_dict[i]=np.log((np.count_nonzero(self.ham_words_list==i) + self.alpha)/(ham_size + self.alpha* all_size))


        

    def predict(self, test_set_x):

        prediction=[]
        all_size=self.all_words_list.size
        ham_size=self.ham_words_list.size
        spam_size=self.spam_words_list.size

        for i in test_set_x:
            # h=np.sum(np.log(self.ham_words_dict[i]))
            # s=np.sum(np.log(self.spam_words_dict[i]))
            c=np.array([np.log(self.prior_spam_prob)])
            v=np.array([np.log(self.prior_ham_prob)])

            for j in i:


                if(j in (self.all_words_list) ):
                    # print(int(self.ham_words_dict[test_set_x[i][j]]))
                    c=np.append(c,self.spam_words_dict[j])
                    v=np.append(v,self.ham_words_dict[j])
                    # s+=np.log(self.spam_words_dict[str(test_set_x[i][j])])
                    # h+=np.log(self.ham_words_dict[str(test_set_x[i][j])])

            prediction.append('ham' if (np.sum(v) > np.sum(c)) else 'spam')


        return np.array(prediction)

## 4 - Training

First of all, we should define a smoothing coeficient (`alpha`).

In [11]:
a = 1

Now we can initialize our model:

In [12]:
model = Naive_Bayes(alpha=a)

Let's train our model:

In [13]:
model.fit(train_set_x, train_set_y)

## 5 - Making predictions

In [14]:
y_predictions = model.predict(test_set_x)

Let's calculate accuracy (accuracy of model must be >0.95):

In [15]:
actual = list(test_set_y)
accuracy = (y_predictions == test_set_y).mean()
print(accuracy)

0.9713004484304932


## 6 - Conclusion
As we can see, our model fits well the hypothesis function to the data.

#### What's next:
1. Try experimenting with the `alpha` to see how this affects the model you have built.
2. Compare the results you have obtained with the `sklearn.naive_bayes.MultinomialNB` model.
3. Try this model in the wild! Select your favorite dataset [here](https://www.kaggle.com/datasets?sortBy=hottest&group=public&page=1&pageSize=20&size=small&filetype=all&license=all&tagids=13303) and play with it.